In [2]:
# Mount Google Drive to access your PDFs
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!apt-get update
!apt-get install -y poppler-utils

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Fetched 261 kB in 2s (142 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pa

In [4]:
!pip3 install colpali-engine vidore_benchmark pdf2image google-generativeai pypdf pyvespa vespacli requests PyPDF2

In [1]:
import os
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from io import BytesIO
import numpy as np
import base64
from io import BytesIO
from PIL import Image
import re
from typing import Dict, List
from binascii import hexlify
import gc
from colpali_engine.models import ColPali, ColPaliProcessor
from colpali_engine.utils.torch_utils import get_torch_device

import pdf2image
from pathlib import Path
from tqdm import tqdm
from PyPDF2 import PdfReader
from pdf2image import convert_from_path
import pytesseract

from vespa.package import Schema, Document, Field, FieldSet
from vespa.package import ApplicationPackage
from vespa.package import RankProfile, Function, FirstPhaseRanking, SecondPhaseRanking
from vespa.deployment import VespaCloud
from vespa.application import Vespa
from vespa.io import VespaResponse
from vespa.package import AuthClient, Parameter


In [ ]:

# Path to your .pem file
pem_file_path = "/content/drive/MyDrive/Colab_Secrets/punya.github.puns.pem"

# Read the contents of the .pem file
with open(pem_file_path, "r") as file:
    pem_contents = file.read()

# Set the contents as an environment variable
os.environ["VESPA_TEAM_API_KEY"] = pem_contents



In [ ]:
# Read the API key from your file
with open('/content/drive/MyDrive/Colab_Secrets/vespa_token.txt', 'r') as file:
    api_key = file.read().strip()

# Set it as an environment variable
os.environ['VESPA_CLOUD_SECRET_TOKEN'] = api_key



In [7]:
colbert_schema = Schema(
    name="doc",
    document=Document(
        fields=[
            Field(
                name="title",
                type="string",
                indexing=["summary"],
            ),
            Field(
                name="texts",
                type="array<string>",
                indexing=["index"],
                index="enable-bm25",
            ),
            Field(
                name="images",
                type="array<string>",
                indexing=["summary"],
            ),
            Field(
                name="colbert",
                type="tensor<int8>(page{}, patch{}, v[16])",
                indexing=["attribute"],
            ),
        ]
    ),
    fieldsets=[FieldSet(name="default", fields=["title", "texts"])],
)

In [8]:
vespa_app_name = "visionrag"
tenant_name = "puns"

vespa_application_package = ApplicationPackage(
    name=vespa_app_name, schema=[colbert_schema]
)

In [9]:
colbert_profile = RankProfile(
    name="default",
    inputs=[("query(qt)", "tensor<float>(querytoken{}, v[128])")],
    functions=[
        Function(
            name="max_sim_per_page",
            expression="""
                sum(
                    reduce(
                        sum(
                            query(qt) * unpack_bits(attribute(colbert)) , v
                        ),
                        max, patch
                    ),
                    querytoken
                )
            """,
        ),
        Function(name="max_sim", expression="reduce(max_sim_per_page, max, page)"),
        Function(name="bm25_score", expression="bm25(texts)"),
    ],
    first_phase=FirstPhaseRanking(expression="bm25_score"),
    second_phase=SecondPhaseRanking(expression="max_sim", rerank_count=10),
    match_features=["max_sim_per_page", "bm25_score"],
)
colbert_schema.add_rank_profile(colbert_profile)

In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

key = os.getenv("VESPA_TEAM_API_KEY", None)
if key is not None:
    key = key.replace(r"\n", "\n")  # To parse key correctly

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=vespa_app_name,
    key_content=key,
    application_package=vespa_application_package,
)

Setting application...
Running: vespa config set application puns.visionrag.default
Setting target cloud...
Running: vespa config set target cloud

Api-key found for control plane access. Using api-key.


In [11]:
app: Vespa = vespa_cloud.deploy()


Deployment started in run 12 of dev-aws-us-east-1c for puns.visionrag. This may take a few minutes the first time.
INFO    [17:31:56]  Deploying platform version 8.509.33 and application dev build 12 for dev-aws-us-east-1c of default ...
INFO    [17:31:56]  Using CA signed certificate version 1
INFO    [17:31:56]  Using 1 nodes in container cluster 'visionrag_container'
INFO    [17:32:00]  Session 62 for tenant 'puns' prepared and activated.
INFO    [17:32:21]  ######## Details for all nodes ########
INFO    [17:32:28]  h113694a.dev.us-east-1c.aws.vespa-cloud.net: expected to be UP
INFO    [17:32:28]  --- platform vespa/cloud-tenant-rhel8:8.509.33
INFO    [17:32:28]  --- container-clustercontroller on port 19050 has not started 
INFO    [17:32:28]  --- metricsproxy-container on port 19092 has not started 
INFO    [17:32:28]  h116695a.dev.us-east-1c.aws.vespa-cloud.net: expected to be UP
INFO    [17:32:28]  --- platform vespa/cloud-tenant-rhel8:8.509.33
INFO    [17:32:28]  --- container

In [12]:
CLIENT_TOKEN_ID = 'api-token'

# Same as token name from the Vespa Cloud Console
auth_clients = [
    AuthClient(
        id="mtls",  # Note that you still need to include the mtls client.
        permissions=["read", "write"],
        parameters=[Parameter("certificate", {"file": "security/clients.pem"})],
    ),
    AuthClient(
        id="token",
        permissions=["read"],
        parameters=[Parameter("token", {"id": CLIENT_TOKEN_ID})],
    ),
]

updated_app_package = ApplicationPackage(
    name=vespa_app_name, schema=[colbert_schema], auth_clients=auth_clients
)

In [13]:
instance = "token"

vespa_cloud = VespaCloud(
    tenant=tenant_name,
    application=vespa_app_name,
    key_content=key,
    application_package=updated_app_package,
)

app = vespa_cloud.deploy(instance=instance)

Setting application...
Running: vespa config set application puns.visionrag.default
Setting target cloud...
Running: vespa config set target cloud

Api-key found for control plane access. Using api-key.
Deployment started in run 10 of dev-aws-us-east-1c for puns.visionrag.token. This may take a few minutes the first time.
INFO    [17:33:44]  Deploying platform version 8.509.33 and application dev build 10 for dev-aws-us-east-1c of token ...
INFO    [17:33:44]  Using CA signed certificate version 1
INFO    [17:33:44]  Using 1 nodes in container cluster 'visionrag_container'
INFO    [17:33:47]  Session 63 for tenant 'puns' prepared, but activation failed: 1/2 application hosts and 2/2 admin hosts for puns.visionrag.token have completed provisioning and bootstrapping, still waiting for h116726.dev.us-east-1c.aws.vespa-cloud.net
INFO    [17:33:48]  Deploying platform version 8.509.33 and application dev build 10 for dev-aws-us-east-1c of token ...
INFO    [17:33:48]  1/2 application hosts 

In [14]:
token_app = vespa_cloud.get_application(
    instance=instance,
    endpoint_type="token",
    vespa_cloud_secret_token=os.getenv("VESPA_CLOUD_SECRET_TOKEN"),
)

Only region: aws-us-east-1c available in dev environment.
Found token endpoint for visionrag_container
URL: https://e7c888f9.e41c6780.z.vespa-app.cloud/
Application is up!


In [15]:
token_app.get_application_status()

<Response [200]>

In [16]:
token_endpoint = vespa_cloud.get_token_endpoint(instance=instance)
token_endpoint

Found token endpoint for visionrag_container
URL: https://e7c888f9.e41c6780.z.vespa-app.cloud/


'https://e7c888f9.e41c6780.z.vespa-app.cloud/'

In [17]:

def clean_text(text):
    # 1. Remove non-printable characters except normal whitespace
    text = ''.join(char for char in text if char.isprintable() or char in ['\n', '\t'])

    # 2. Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)

    # 3. Remove Unicode control characters and zero-width characters
    text = re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F-\x9F\uFEFF\uFFFE\uFFFF]', '', text)

    # 4. Replace common problematic Unicode characters with their ASCII equivalents
    replacements = {
        '"': '"',
        '"': '"',
        ''': "'",
        ''': "'",
        '–': '-',
        '—': '-',
        '…': '...'
    }
    for old, new in replacements.items():
        text = text.replace(old, new)

    # 5. Strip leading/trailing whitespace
    text = text.strip()

    return text

In [18]:
from PIL import Image

def scale_image(image, max_width):
    """
    Scales an image to the specified maximum width while maintaining the aspect ratio.

    Args:
        image (PIL.Image.Image): The image to be scaled.
        max_width (int): The maximum width of the scaled image.

    Returns:
        PIL.Image.Image: The scaled image.
    """
    # Get the original dimensions of the image
    original_width, original_height = image.size

    # Calculate the scaling factor to maintain the aspect ratio
    scaling_factor = max_width / float(original_width)

    # Calculate the new height based on the scaling factor
    new_height = int(original_height * scaling_factor)

    # Resize the image with the new dimensions
    scaled_image = image.resize((max_width, new_height), Image.Resampling.LANCZOS)

    return scaled_image

In [19]:
def get_base64_image(image, add_url_prefix=True):
    buffered = BytesIO()
    image.save(buffered, format="PNG")  # Save the image to a BytesIO object
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")  # Encode to Base64
    if add_url_prefix:
        return f"data:image/png;base64,{img_str}"  # Add URL prefix for HTML embedding
    return img_str

In [20]:
# Check if we have GPU
print(f"GPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU model: {torch.cuda.get_device_name(0)}")

GPU available: True
GPU model: NVIDIA L4


In [21]:
torch.cuda.empty_cache()

In [22]:
device = get_torch_device("auto")
if device == "cuda":
    dtype = torch.bfloat16
else:
    dtype = torch.float32

In [23]:
model_name = "vidore/colpali-v1.2"
model = ColPali.from_pretrained(model_name, torch_dtype=dtype, device_map=device).eval()
processor = ColPaliProcessor.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [24]:
def get_pdf_images(pdf_files, folder_path, limit_pdfs=None):
    """
    Reads PDFs from a local folder, extracts images and text for each page.

    :param folder_path: Path to the folder that contains PDF files.
    :param limit_pdfs: Optional integer specifying how many PDFs to read.
                       If None, reads all PDFs in the folder.
    :return: A dictionary with keys as the PDF file names and values as tuples of (title, images, page_texts).
    """


    # If limit_pdfs is specified, truncate the list
    if limit_pdfs is not None:
        pdf_files = pdf_files[:limit_pdfs]

    results = {}

    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        images = convert_from_path(pdf_path)
        page_texts = []
        for page_number, image in enumerate(images):
            # Perform OCR on the image to extract text
            text = pytesseract.image_to_string(image)
            text = clean_text(text)  # Assuming clean_text is a custom function for text cleaning
            page_texts.append(text)


        # Extract title from the PDF file name (remove .pdf extension)
        title = os.path.splitext(pdf_file)[0]

        # Store in dictionary keyed by the PDF file name
        results[pdf_file] = (title, images, page_texts)

    return results



In [25]:
pdf_path = "/content/drive/MyDrive/Projects/RagTaxHelper/irs_eng_pdfs"

In [26]:
def binarize_token_vectors_hex(vectors: List[torch.Tensor]) -> Dict[str, str]:
    vespa_tensor = list()
    for page_id in range(0, len(vectors)):
        page_vector = vectors[page_id]
        binarized_token_vectors = np.packbits(
            np.where(page_vector > 0, 1, 0), axis=1
        ).astype(np.int8)
        for patch_index in range(0, len(page_vector)):
            values = str(
                hexlify(binarized_token_vectors[patch_index].tobytes()), "utf-8"
            )
            if (
                values == "00000000000000000000000000000000"
            ):  # skip empty vectors due to padding of batch
                continue
            vespa_tensor_cell = {
                "address": {"page": page_id, "patch": patch_index},
                "values": values,
            }
            vespa_tensor.append(vespa_tensor_cell)

    return vespa_tensor

In [27]:
def process_pdfs_in_batches(folder_path, batch_size):
    """
    Processes PDF files in a folder in smaller batches.

    Args:
        folder_path (str): Path to the folder containing PDF files.
        batch_size (int): Number of PDFs to process in each batch.
        process_function (function): A function that processes a list of PDF files.
                                     This function should accept a list of file paths as input.
    """
    # Get the list of PDF files in the folder
    pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]
    pdf_files.sort()  # Sort the files if needed

    # Divide the list into batches
    for i in range(0, len(pdf_files), batch_size):

        batch = pdf_files[i:i + batch_size]
        print(f"Processing batch {i // batch_size + 1}")

        try:
            # Call the processing function on the current batch
            results = get_pdf_images(batch,folder_path)
            print("succesfully processed the pdfs to images")
        except Exception as e:
              print(f"Error processing batch {i // batch_size + 1} in processing pdfs: {e}")

        try :
            #create the embeddings

            for pdf_file, (title, images, texts) in results.items():
                page_embeddings = []

                # Create a DataLoader for the images
                dataloader = DataLoader(
                    images,  # Access images directly from the tuple
                    batch_size=4,
                    shuffle=False,
                    collate_fn=lambda x: processor.process_images(x),
                )

                for batch_doc in tqdm(dataloader):
                    with torch.no_grad():
                        batch_doc = {k: v.to(model.device) for k, v in batch_doc.items()}
                        embeddings_doc = model(**batch_doc)
                        # Handle if running on GPU
                        if model.device == "cuda":
                            embeddings_doc = embeddings_doc.float()
                        page_embeddings.extend(list(torch.unbind(embeddings_doc.cpu())))

                # Store the embeddings in the results dictionary
                results[pdf_file] = (title, images, texts, page_embeddings)  # Update the tuple to include embeddings
            print("succesfully created page embeddings")
        except Exception as e:
            print(f"Error processing batch {i // batch_size + 1} in creating page embeddings: {e}")

        vespa_feed = []
        try :
            # Iterate over the results dictionary
            for pdf_file, (title, images, texts, embeddings) in results.items():
                images_base_64 = []

                # Convert each image to base64
                for image in images:
                    images_base_64.append(get_base64_image(image, add_url_prefix=False))

                # Create the document structure for Vespa
                doc = {
                    "fields": {
                        "title": title,  # Include the title
                        "images": images_base_64,
                        "texts": texts,  # Array of text per page
                        "colbert": {  # Colbert embeddings per page
                            "blocks": binarize_token_vectors_hex(embeddings)  # Use the embeddings directly
                        },
                    }
                }

                vespa_feed.append(doc)
            print("successfully created vespa feed")
        except Exception as e:
            print(f"Error processing batch {i // batch_size + 1} in creating vespa feed: {e}")

        # Now vespa_feed contains the documents ready to be sent to Vespa

        try :
            with app.syncio() as sync:
                for operation in vespa_feed:
                    fields = operation["fields"]
                    response: VespaResponse = sync.feed_data_point(
                        data_id=fields["title"], fields=fields, schema="doc"
                    )
                    if not response.is_successful():
                        print(response.json())
            print("successfully synced feed with vespa db")
        except Exception as e:
            print(f"Error processing batch {i // batch_size + 1} in sending data to the vespa app: {e}")

        del batch, results, vespa_feed
        gc.collect()

In [ ]:
process_pdfs_in_batches(pdf_path, 100)

Processing batch 1
succesfully processed the pdfs to images


100%|██████████| 1/1 [00:03<00:00,  3.42s/it]


succesfully created pageembeddings


100%|██████████| 7/7 [00:25<00:00,  3.71s/it]


succesfully created pageembeddings


...

100%|██████████| 3/3 [00:08<00:00,  2.86s/it]


succesfully created pageembeddings


100%|██████████| 37/37 [02:17<00:00,  3.70s/it]


succesfully created pageembeddings


 17%|█▋        | 4/24 [00:14<01:14,  3.74s/it]